In [1]:
import pandas as pd
import numpy as np

In [2]:
import konlpy

In [3]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Twitter

In [4]:
from tqdm.notebook import tqdm
import os

In [5]:
df = pd.read_csv("C:/Users/User/Desktop/주제분석_데이터파일/news_data_final.csv")
df['date'] = pd.to_datetime(df['date'])

In [6]:
df.head()

,Unnamed: 0,date,press,title,content,link
0,0,2021-04-01,국민일보,횡보하더니…비트코인 7300만원 뚫었다,가상화폐 대표주자 비트코인의 가격이 연일 고점을 높이고 있다. 사상 처음으로 7...,http://news.kmib.co.kr/article/view.asp?arcid=...
1,1,2021-04-01,연합뉴스언론사 선정,"비트코인, 고점 계속 높여…7천300만원 돌파",국내 거래소에서 가상화폐 비트코인이 연일 고점을 높이면서 사상 처음으로 7천30...,http://yna.kr/AKR20210401196100002?did=1195m
2,2,2021-04-01,한스경제,"파이코인, '제2의 비트코인' 별칭 갑론을박","◆ 파이코인 지지자들, ""제 2의 비트코인 될 것"" 파이코인은 최근 1500만명...",http://www.sporbiz.co.kr/news/articleView.html...
3,3,2021-04-01,세계일보,7200만원 돌파한 비트코인… 개인투자자 주식보유액은 240조원 증가,국민참여형 뉴딜 펀드 완판 행진 비트코인 시세가 또다시 최고가를 경신했다. 가상...,http://www.segye.com/content/html/2021/04/01/2...
4,4,2021-04-01,더게임스데일리,하이브리드스쿼드 '포기하지마 바운시' 스위치 발매,이 작품은 리듬에 맞춰 적절한 타이밍에 음악 비트의 도움을 받아 점프하는 방식으...,http://www.tgdaily.co.kr/news/articleView.html...


In [7]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'+'↑↓▼▲◆·'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [8]:
def clean_punc(text, punct, mapping): 
    for p in mapping: 
        text = text.replace(p, "") 
    for p in punct: 
        text = text.replace(p, "")
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials: 
        text = text.replace(s, "") 
    return text.strip()

In [9]:
f = open("C:/Users/User/Desktop/주제분석/neg_pol_word.txt",'r',encoding = 'utf-8') # 군산대 부정어

neg_lines = f.readlines()

neg_lines = neg_lines[19:]
for i in range(len(neg_lines)) : 
    neg_lines[i] = neg_lines[i].strip()
    
f.close()

In [10]:
f = open("C:/Users/User/Desktop/주제분석/pos_pol_word.txt",'r',encoding = 'utf-8') # 군산대 긍정어

pos_lines = f.readlines()

pos_lines = pos_lines[19:]
for i in range(len(pos_lines)) : 
    pos_lines[i] = pos_lines[i].strip()
    
f.close()

In [11]:
f = open("C:/Users/User/Desktop/주제분석/긍정어_논문.txt",'r',encoding = 'utf-8') # 논문 긍정어 파일 경로만 바꾸면 새로운 파일 불러들일 수 있음

bit_pos_lines = f.read().split(',')
bit_pos = []

for word in bit_pos_lines : 
    case = ''
    for item in word : 
        if item == '\n': 
            continue 
        else : 
            case += item 
    bit_pos.append(case.strip())
f.close()

In [12]:
f = open("C:/Users/User/Desktop/주제분석/부정어_논문.txt",'r',encoding = 'utf-8') # 논문 부정어

bit_neg_lines = f.read().split(',')
bit_neg = []

for word in bit_neg_lines : 
    case = ''
    for item in word : 
        if item == '\n': 
            continue 
        else : 
            case += item 
    bit_neg.append(case.strip())
f.close()

In [13]:
us_pos = pd.read_csv("C:/Users/User/Desktop/주제분석/긍정어_우리.csv", encoding='utf-8', header=None)

In [14]:
us_neg = pd.read_csv("C:/Users/User/Desktop/주제분석/부정어_우리.csv", encoding='utf-8', header=None)

In [15]:
our_pos = us_pos.iloc[:,0].tolist()
our_neg = us_neg.iloc[:,0].tolist()

In [16]:
okt = Okt()

In [17]:
important_list = ["Noun","Verb", "Adjective",'Adverb']

new_df = df

pos = our_pos + bit_pos     # bit_pos는 논문, pos_lines는 군산대 
neg = our_neg + bit_neg     # 얘도 마찬가지(그냥 +로 추가해주면 됩니당 다른 사전도)
pos_list = []
neg_list = []
for i in tqdm(range(df.shape[0])): 
    pos_num = 0
    neg_num = 0
    for word in okt.pos(clean_punc(df['content'][i], punct, punct_mapping),stem = True) : 
    
        if word[1] in important_list : 
            if word[0] in pos : 
                pos_num += 1
            elif word[0] in neg : 
                neg_num += 1
    pos_list.append(pos_num)
    neg_list.append(neg_num)


In [18]:
new_df['pos'] = pd.Series(pos_list)
new_df['neg'] = pd.Series(neg_list)

In [19]:
len(pos_list)

357957

In [20]:
os.chdir("C:/Users/User/Desktop/주제분석")
new_df.to_csv("긍정부정단어세기(2).csv", encoding ='utf-8', index = False)